In [ ]:
#start by importing modules
#enable interactive use of figures
%matplotlib widget
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

plt.ion() # make the plotting interactive


In [ ]:
def generate_handles(labels, colors, edge='k', alpha=1):
    """
    Generate matplotlib patch handles to create a legend of each of the features in the map.

    Parameters
    ----------

    labels : list(str)
        the text labels of the features to add to the legend

    colors : list(matplotlib color)
        the colors used for each of the features included in the map.

    edge : matplotlib color (default: 'k')
        the color to use for the edge of the legend patches.

    alpha : float (default: 1.0)
        the alpha value to use for the legend patches.

    Returns
    -------

    handles : list(matplotlib.patches.Rectangle)
        the list of legend patches to pass to ax.legend()
    """
    lc = len(colors)  # get the length of the color list
    handles = [] # create an empty list
    for ii in range(len(labels)): # for each label and color pair that we're given, make an empty box to pass to our legend
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[ii % lc], edgecolor=edge, alpha=alpha))
    return handles

In [ ]:
# here we are defining a function to create a scale bar
# adapted this question: https://stackoverflow.com/q/32333870
# answered by SO user Siyh: https://stackoverflow.com/a/35705477
def scale_bar(ax, length=10, location=(0.92, 0.95)):
    """
    Create a scale bar in a cartopy GeoAxes.

    Parameters
    ----------

    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes to add the scalebar to.

    length : int, float (default 20)
        the length of the scalebar, in km

    location : tuple(float, float) (default (0.92, 0.95))
        the location of the center right corner of the scalebar, in fractions of the axis.

    Returns
    -------
    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes object

    """
    x0, x1, y0, y1 = ax.get_extent() # get the current extent of the axis
    sbx = x0 + (x1 - x0) * location[0] # get the right x coordinate of the scale bar
    sby = y0 + (y1 - y0) * location[1] # get the right y coordinate of the scale bar

    ax.plot([sbx, sbx-length*1000], [sby, sby], color='k', linewidth=4, transform=ax.projection) # plot a thick black line
    ax.plot([sbx-(length/2)*1000, sbx-length*1000], [sby, sby], color='w', linewidth=2, transform=ax.projection) # plot a white line from 0 to halfway

    ax.text(sbx, sby-(length/4)*1000, f"{length} km", ha='center', transform=ax.projection, fontsize=6) # add a label at the right side
    ax.text(sbx-(length/2)*1000, sby-(length/4)*1000, f"{int(length/2)} km", ha='center', transform=ax.projection, fontsize=6) # add a label in the center
    ax.text(sbx-length*1000, sby-(length/4)*1000, '0 km', ha='center', transform=ax.projection, fontsize=6) # add a label at the left side

    return ax

In [ ]:
#  this cell defines a function to reproject the data to the correct projected CRS on the fly if needed but best practice to ensure data using correct CRS prior to reading in
#
#def set_and_reproj(gdf, current_epsg=4326, target_epsg=32430):
    '''
    Assigns current epsg to a GeoDataFrame if missing, then reprojects it to target CRS.

    Parameters:
    - gdf: GeoDataFrame
    - current_epsg: EPSG code of current CRS (default: 4326)
    - target_epsg: EPSG code to reproject to (default: 32430)

    Returns:
    - Reprojected GeoDataFrame
    '''
    #if gdf.crs is None:
        gdf = gdf.set_crs(epsg=current_epsg)

    #return gdf.to_crs(epsg=target_epsg)

# read in data and use function to reproject, skip this step and move onto #loading datasets below if files are using correct CRS
#surveyarea = set_and_reproj(gpd.read_file('data/survey_area.shp'), current_epsg=4326)
#habitat = set_and_reproj(gpd.read_file('data/habitat3.shp'), current_epsg=4326)
#birds = set_and_reproj(gpd.read_file('data/birdrecord.shp'), current_epsg=4326)

In [ ]:
# load in the datasets
surveyarea = gpd.read_file(os.path.abspath('data/surveyarea4.shp'))
habitat = gpd.read_file(os.path.abspath('data/habitat4.shp'))
birds = gpd.read_file(os.path.abspath('data/bird_record2.shp'))

In [ ]:
scot_utm = ccrs.UTM(30)  # this creates a Universal Transverse Mercator reference system to transform our data.
# Scotland is in UTM Zone 30, so we pass 30 to ccrs.UTM()

In [ ]:
ccrs.CRS(surveyarea.crs) # here we create a cartopy CRS representation of the CRS associated with the surveyarea dataset

In [ ]:
ccrs.CRS(habitat.crs)

In [ ]:
ccrs.CRS(birds.crs)

In [ ]:
#plot data individually to ensure all using same CRS
ax = surveyarea.boundary.plot()
habitat.boundary.plot(ax=ax)
birds.plot(ax=ax)





In [ ]:
fig = plt.figure(figsize=(8, 8))  # create a figure of size 8x8 (representing the page size in inches)
ax = plt.axes(projection=scot_utm)  # create an axes object in the figure, using the defined UTM projection within which we can actually plot our data.

In [ ]:
# first add the surveyarea dataset using cartopy's ShapelyFeature
surveyarea_feature = ShapelyFeature(surveyarea['geometry'], scot_utm, edgecolor='k', facecolor='w')
ax.add_feature(surveyarea_feature) # this adds the features we've created to the map.

In [ ]:
xmin, ymin, xmax, ymax = surveyarea.total_bounds
# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-4000, xmax+4000, ymin-4000, ymax+4000], crs=scot_utm)  # because total_bounds
# gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.
fig

In [ ]:
# select colors and then add features to the map
habitat_colors = ['#003f5c', '#444e86', '#955196', '#dd5182', '#ff6e54', '#ffa600']

# get a list of unique names for the habitat types
habitat_type = list(habitat['type'])
habitat_type.sort() # sort the counties alphabetically by name

In [ ]:
# next step is to add the habitat polygons to the map using the colors selected above
# here, we're iterating over the unique values in the 'type' field.
# we're also setting the edge color to be black, with a line width of 0.5 pt.

for ii, name in enumerate(habitat_type):
    feat = ShapelyFeature(habitat.loc[habitat['type'] == name, 'geometry'], # first argument is the geometry
                          ccrs.CRS(habitat.crs), # second argument is the CRS
                          edgecolor='k', # outline the feature in black
                          facecolor=habitat_colors[ii], # set the face color to the corresponding color from the list
                          linewidth=1, # set the outline width to be 1 pt
                          alpha=0.6) # set the alpha (transparency) from a scale up to 1
    ax.add_feature(feat) # once we have created the feature, we have to actively add it to the map using ax.add_feature()

In [ ]:
# here, we're setting the edge color to be the same as the face color. Feel free to change this around,
# and experiment with different line widths.
#route_feat = ShapelyFeature(route['geometry'], # first argument is the geometry
                            #ccrs.CRS(route.crs), # second argument is the CRS
                            #edgecolor='mediumblue', # set the edgecolor to be mediumblue
                            #facecolor='mediumblue', # set the facecolor to be mediumblue
                            #linewidth=1) # set the outline width to be 1 pt
#ax.add_feature(route_feat) # add the collection of features to the map

In [ ]:
#route_feat = ShapelyFeature(route['geometry'], # first argument is the geometry
                            #ccrs.CRS(route.crs), # second argument is the CRS
                            #edgecolor='mediumblue', # set the edgecolor to be royalblue
                           # linewidth=0.2) # set the linewidth to be 0.2 pt
#ax.add_feature(route_feat) # add the collection of features to the map

In [ ]:
# ShapelyFeature creates a polygon, so when working with point data we can just use ax.plot()
# here we add the bird point records
birds_handle = ax.plot(birds.geometry.x, birds.geometry.y, 's', color='0.5', ms=4, transform=ccrs.CRS(birds.crs))
fig

In [ ]:
# generate a list of handles for the habitat datasets
# first, add the list of habitat type, then the list of colors, and finally set the transparency

habitat_handles = generate_handles(habitat.type.unique(), habitat_colors, alpha=0.8)

In [ ]:
# ax.legend() takes a list of handles and a list of labels corresponding to the objects
# here we add to the legend
handles = habitat_handles + birds_handle # use '+' to concatenate (combine) lists
labels =  ['Habitat', 'Birds']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=8,
                 fontsize=5, loc='upper left', frameon=True, framealpha=1)
fig

In [ ]:
 # draw labels for the grid lines
gridlines = ax.gridlines(draw_labels=True,)                        
gridlines.left_labels = False # turn off the left-side labels
gridlines.bottom_labels = False # turn off the bottom labels

In [ ]:
# add the text labels for the bird records
for ind, row in birds.iterrows(): # birds.iterrows() returns the index and row
    x, y = row.geometry.x, row.geometry.y # get the x,y location for each bird record
    ax.text(x, y, row['bird_spp'].title(), fontsize=4, transform=ccrs.CRS(birds.crs)) # use plt.text to place a label at x,y


In [ ]:
# add the scale bar to the axis
scale_bar(ax)

In [ ]:
# final step is to save the figure as SurveyMap.png, cropped to the axis (bbox_inches='tight'), and a dpi of 200
fig.savefig('SurveyMap.png', bbox_inches='tight', dpi=200)